In [5]:
import pandas as pd
import zipfile # it is used for reading the zip file without extrating
import numpy as np
import matplotlib.pyplot as plt

In [7]:
#reading files

with zipfile.ZipFile("matches.zip") as Z:
    with Z.open("matches_England.json") as f:
        matches_df = pd.read_json(f)
        
with zipfile.ZipFile("events.zip") as Z:
    with Z.open("events_England.json") as f:
        events_df = pd.read_json(f)
        
competitions = pd.read_json("competitions.json")
playerRank = pd.read_json("playerank.json")
players = pd.read_json("players.json")

eventId_name = pd.read_csv("eventid2name.csv")
tags_name = pd.read_csv("tags2name.csv")

# all_teams

In [8]:
# matches_df.head()

In [9]:
all_teams =pd.read_json("teams.json")

In [10]:
# events_df.shape, events_df.head()

In [11]:

# competitions

In [12]:

# playerRank.shape,playerRank.head()

In [13]:
# players.head()

In [14]:

# tags_name

In [15]:
# matches_df

In [16]:
# events_df

In [17]:
# matches_df[matches_df.wyId == 2499719]

In [18]:
#weekly points

point_data = list()
for i in range(len(matches_df)):
    gameweek = matches_df.iloc[i].gameweek# iloc is used for the locating data in a data frame
    label = matches_df.iloc[i].label
    match_id = matches_df.iloc[i].wyId
    
    score = label.split(",")[1]
    b = score.split("-")
    c = [int(i) for i in b]
    diff = c[0] - c[1]
    
    [[home_team, away_team], [home_score, away_score]] = [[o.strip() for o in s.split('-')] for s in label.split(',')]# here we have converted the label column into a nested list, od home team/away team . HOme score and Away score 
    home_score = int(home_score)
    away_score = int(away_score)
    if home_score > away_score:# we have identified the winner /looser and Draw
        home_point = 3
        away_point = 0
    if away_score > home_score:
        away_point = 3
        home_point = 0
    if away_score == home_score:
        home_point = 1
        away_point = 1
    point_data.append([gameweek, home_team, home_point,label,match_id,diff]) #here we have summarize the team with week and point all together
    point_data.append([gameweek, away_team, away_point,label,match_id,-diff])
point_df = pd.DataFrame(point_data, columns=['gameweek', 'team', 'point','label','matchId','tempgd']) #setup a datafram

In [19]:
weeks = np.sort(point_df.gameweek.unique())
teams = point_df.team.unique()
points_cp = pd.DataFrame([])

for team in teams:
    t1 = point_df[point_df.team == team]
    cumP = {}
    cumgd ={}
    t1 = t1.sort_values(by=['gameweek'])
    for week in weeks:
        if week == 1:
            cumP[week] = int(t1[t1.gameweek == 1]['point'])
            cumgd[week] = int(t1[t1.gameweek == 1]['tempgd'])
        else:
            cumP[week] = int(t1[t1.gameweek == week]['point']) + cumP.get(week -1) 
            cumgd[week] = int(t1[t1.gameweek == week]['tempgd']) + cumgd.get(week -1)
    
    t1['cummPoints'] = t1.apply(lambda x : cumP.get(x.gameweek),axis =1)
    t1['gd'] = t1.apply(lambda x : cumgd.get(x.gameweek),axis =1)
#     print(t1)
    points_cp = points_cp.append(t1)

In [20]:
weekly_points = pd.DataFrame()
for week in weeks:
    t1 = points_cp[points_cp.gameweek == week]
    t2 =t1.sort_values(by=['cummPoints','gd'],ascending=False)
    t2['rank'] = [i for i in range(1,t2.shape[0]+1)]
    weekly_points = weekly_points.append(t2)

In [21]:
weekly_points

,gameweek,team,point,label,matchId,tempgd,cummPoints,gd,rank
740,1,Manchester United,3,"Manchester United - West Ham United, 4 - 0",2499724,4,3,4,1
749,1,Huddersfield Town,3,"Crystal Palace - Huddersfield Town, 0 - 3",2499722,3,3,3,2
745,1,Manchester City,3,"Brighton & Hove Albion - Manchester City, 0 - 2",2499720,2,3,2,3
743,1,Tottenham Hotspur,3,"Newcastle United - Tottenham Hotspur, 0 - 2",2499725,2,3,2,4
747,1,Burnley,3,"Chelsea - Burnley, 2 - 3",2499721,1,3,1,5
...,...,...,...,...,...,...,...,...,...
4,38,Huddersfield Town,0,"Huddersfield Town - Arsenal, 0 - 1",2500091,-1,37,-30,16
12,38,Southampton,0,"Southampton - Manchester City, 0 - 1",2500095,-1,36,-19,17
14,38,Swansea City,0,"Swansea City - Stoke City, 1 - 2",2500096,-1,33,-28,18
15,38,Stoke City,3,"Swansea City - Stoke City, 1 - 2",2500096,1,33,-33,19


In [22]:
weekly_points = weekly_points.sort_values(by=["team",'gameweek'])

In [23]:
weekly_points

,gameweek,team,point,label,matchId,tempgd,cummPoints,gd,rank
755,1,AFC Bournemouth,0,"West Bromwich Albion - AFC Bournemouth, 1 - 0",2499728,-1,0,-1,13
728,2,AFC Bournemouth,0,"AFC Bournemouth - Watford, 0 - 2",2499729,-2,0,-3,16
718,3,AFC Bournemouth,0,"AFC Bournemouth - Manchester City, 1 - 2",2499739,-1,0,-4,18
689,4,AFC Bournemouth,0,"Arsenal - AFC Bournemouth, 3 - 0",2499749,-3,0,-7,19
678,5,AFC Bournemouth,3,"AFC Bournemouth - Brighton & Hove Albion, 2 - 1",2499759,1,3,-6,19
...,...,...,...,...,...,...,...,...,...
92,34,West Ham United,1,"West Ham United - Stoke City, 1 - 1",2500058,0,36,-18,13
77,35,West Ham United,0,"Arsenal - West Ham United, 4 - 1",2500060,-3,36,-21,14
56,36,West Ham United,0,"West Ham United - Manchester City, 1 - 4",2500078,-3,36,-24,15
43,37,West Ham United,3,"Leicester City - West Ham United, 0 - 2",2500084,2,39,-22,15


In [24]:
weekly_points.to_csv("englandLeague.csv")

In [20]:
teams

array(['Burnley', 'AFC Bournemouth', 'Crystal Palace',
       'West Bromwich Albion', 'Huddersfield Town', 'Arsenal',
       'Liverpool', 'Brighton & Hove Albion', 'Manchester United',
       'Watford', 'Newcastle United', 'Chelsea', 'Southampton',
       'Manchester City', 'Swansea City', 'Stoke City',
       'Tottenham Hotspur', 'Leicester City', 'West Ham United',
       'Everton'], dtype=object)